In [50]:
import json
import os
import re
from pathlib import Path
import glob
from collections import OrderedDict
from operator import itemgetter
import math

# for all the files in folder and subfolders, if files ends with "_results" print the file name
folder = r"C:\\OneDrive_CIRAD\\OneDrive - Cirad\\ML"
name_list = []
full_path_list = []
for path, subdirs, files in os.walk(folder):
    for name in files:
        if name.endswith("_results.json"):
            # if there is a subdirectory with the same name as the file without _results.json
            # then print the path of the file
            name_wo_time = name[:-13]
            if "RMSE" in name_wo_time:
                canonical_name = "_".join(name_wo_time.split("_")[:-1])
            else:
                canonical_name = name_wo_time

            if os.path.isdir(os.path.join(path, name_wo_time)) and "Paprica" not in canonical_name and "Raisin" not in canonical_name:
                name_list.append(canonical_name)
                full_path_list.append(
                    (os.path.join(path, name), canonical_name))
            else:
                pass
                # print("No folder for", canonical_name)


def clean_result(result, path, name):
    if "targetRMSE" in result:
        del result["targetRMSE"]
    keys = ["seed", "RMSE", "MAE", "MAPE", "R2", "MSE", "MedAE", "EVS"]
    for k in keys:
        if k in result:
            try:
                if k == "seed":
                    result[k] = int(result[k])
                else:
                    result[k] = float(result[k])
            except TypeError as e:
                print(e)
                print(k, result[k])
    try:
        close = math.isclose(result["RMSE"], (result["MSE"]*result["MSE"]))
    except:
        print(result, path, name)

    if close:
        tmp = result["RMSE"]
        result["RMSE"] = result["MSE"]
        result["MSE"] = tmp
        # print("inverted RMSE and MSE for", result["name"])

    return result


results = {}
for path, name in full_path_list:
    if name not in results:
        results[name] = []
    json_results = json.load(open(path))
    if isinstance(json_results, list):
        for result in json_results:
            if isinstance(result, dict):
                results[name].append(clean_result(result, path, name))
            elif isinstance(result, list):
                for r in result:
                    results[name].append(clean_result(r, path, name))
            else:
                print("unknown type", type(result))
    else:
        for k, v in json_results.items():
            if isinstance(v, dict):
                results[name].append(clean_result(v, path, name))
            elif isinstance(v, list):
                for r in v:
                    results[name].append(clean_result(r, path, name))
            else:
                print("unknown type", type(v))

for k, v in results.items():
    print(k, v)
    # break
    results[k] = sorted(v, key=itemgetter("RMSE"))

json.dump(results, open("all_results.json", "w"), indent=4)

decon-haar-31441 C:\\OneDrive_CIRAD\\OneDrive - Cirad\\ML\results\BAK\ALPINE_Calpine_424_Murguzur_results.json ALPINE_Calpine_424_Murguzur


UnboundLocalError: local variable 'close' referenced before assignment